In [2]:
import sklearn.datasets as skds
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model

Using TensorFlow backend.


In [5]:
file_train=pd.read_csv('train.csv')#paste your path

In [6]:
file_train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
train,test=train_test_split(file_train,test_size=0.2,random_state=2)

In [5]:
x_train=train['tweet']
y_train=train['label']
x_test=test['tweet']
y_test=test['label']

In [ ]:
y_train = pd.get_dummies(y_train)

In [ ]:
y_test2 = pd.get_dummies(y_test)

In [2]:
import nltk
#nltk.download('punkt')

In [7]:
sent_lens=[]
for sent in file_train['tweet']:
    sent_lens.append(len(word_tokenize(sent)))

In [8]:
#maximum length of reviews in data set
max(sent_lens)

75

In [9]:
np.quantile(sent_lens,0.95)

48.0

In [10]:
max_len = 50
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(x_train)
sequences_train = tok.texts_to_sequences(x_train)

In [11]:
sequences_train

[[245, 4, 39, 797, 8, 873, 1627, 2987, 6054, 26, 8, 223, 2104, 6055],
 [296, 8, 15, 4, 127, 3861, 6, 99, 2105, 3862, 138, 4],
 [1827,
  36,
  54,
  1827,
  380,
  837,
  54,
  1828,
  6056,
  11,
  692,
  6057,
  6058,
  2,
  7,
  3,
  5,
  6059],
 [332, 1331, 6060, 1, 138, 562, 381, 381, 381, 54, 19, 7, 3, 5, 6061],
 [321,
  31,
  301,
  71,
  286,
  1829,
  595,
  1628,
  2,
  619,
  63,
  6062,
  6063,
  147,
  76,
  139,
  1332],
 [119,
  21,
  4,
  8,
  912,
  435,
  151,
  127,
  405,
  256,
  26,
  6,
  20,
  9,
  436,
  2106,
  21,
  119,
  21,
  4,
  3863,
  3864],
 [1150,
  9,
  1017,
  6,
  20,
  10,
  49,
  1,
  165,
  261,
  18,
  1629,
  38,
  49,
  1830,
  6064,
  2988,
  3865,
  4],
 [6065,
  6066,
  6067,
  21,
  6068,
  183,
  10,
  6069,
  2989,
  444,
  65,
  50,
  466,
  10,
  510,
  21,
  1,
  120,
  2990,
  436,
  92],
 [106, 22, 201, 1080, 148, 6070, 1, 581, 6071, 406, 2, 25, 23, 5, 6072],
 [6,
  1,
  1630,
  3866,
  1631,
  1831,
  9,
  162,
  185,
  3867,
  12

In [12]:
tok.index_word

{1: 'iphone',
 2: 'http',
 3: 'com',
 4: 'apple',
 5: 'p',
 6: 'my',
 7: 'instagram',
 8: 'the',
 9: 'i',
 10: 'to',
 11: 'samsung',
 12: 'a',
 13: 'and',
 14: 'twitter',
 15: 'new',
 16: 'for',
 17: 'me',
 18: 'it',
 19: 'https',
 20: 'phone',
 21: 'you',
 22: 'is',
 23: 'am',
 24: 'sony',
 25: 'instagr',
 26: 'on',
 27: 'of',
 28: 'in',
 29: 'follow',
 30: '…',
 31: 'this',
 32: 'www',
 33: 'with',
 34: 'ipad',
 35: 'pic',
 36: 'so',
 37: 'android',
 38: 'have',
 39: 'just',
 40: 'at',
 41: 'love',
 42: 'life',
 43: 'like',
 44: 'now',
 45: 'all',
 46: 'ly',
 47: 'rt',
 48: 'day',
 49: 'an',
 50: 'your',
 51: 'that',
 52: 'photography',
 53: 'photo',
 54: 'cute',
 55: 'not',
 56: 'case',
 57: 'today',
 58: 'get',
 59: 'gain',
 60: 'galaxy',
 61: 'back',
 62: 'app',
 63: 'be',
 64: 'got',
 65: 'from',
 66: 'news',
 67: 'bit',
 68: 'fun',
 69: 'music',
 70: 'itunes',
 71: 'out',
 72: 'ios',
 73: 'happy',
 74: "i'm",
 75: 'time',
 76: 'funny',
 77: 'are',
 78: 'instagood',
 79: 'who',
 

In [13]:
vocab_len=len(tok.index_word.keys())

In [14]:
vocab_len

20602

In [15]:
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

In [16]:
sequences_matrix_train

array([[    0,     0,     0, ...,   223,  2104,  6055],
       [    0,     0,     0, ...,  3862,   138,     4],
       [    0,     0,     0, ...,     3,     5,  6059],
       ...,
       [    0,     0,     0, ..., 20596,   402,    30],
       [    0,     0,     0, ...,   310,    46, 20599],
       [    0,     0,     0, ...,     2, 20602,     3]])

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(vocab_len+1,500,input_length=max_len,mask_zero=True)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(2,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [17]:
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [ ]:
model.fit(sequences_matrix_train,y_train,batch_size=50,epochs=50,validation_data=(sequences_matrix_test,y_test2))

In [ ]:
model.save('tweet_model.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

In [18]:
from keras.models import load_model
# returns a compiled model
# identical to the previous one
model = load_model('tweet_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [19]:
predictions=model.predict(sequences_matrix_test)

In [20]:
predictions = np.argmax(predictions,1)

In [21]:
from sklearn.metrics import roc_auc_score

In [22]:
roc_auc_score(y_test,predictions)

0.8180978494979954